In [2]:
from pycorese.api import CoreseAPI

python_to_java_bridge = 'py4j'
corese = CoreseAPI(java_bridge=python_to_java_bridge)
_ = corese.loadCorese()

2025-04-12 18:30:42,576 - INFO - Py4J: Loading CORESE...
2025-04-12 18:30:43,343 - INFO - Py4J: CORESE is loaded


In [3]:
g = corese.loadRDF("../data/graph_data.rdf")

In [4]:
# Simple first 5 triples from the whole graph
res = corese.sparqlSelect(g, "SELECT * WHERE { ?s ?p ?o } LIMIT 5", return_dataframe=True)
res

,s,p,o
0,_:b0,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#managedBy
1,_:b1,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#manages...
2,_:b2,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#item
3,_:b3,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#country
4,_:b4,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#Country


In [5]:
# Since we haven't run ontological reasoning yet, the "manages" relationships are not inferred
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,s,p,o
0,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#CHRI...
1,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#BIRT...
2,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#APPAREL
3,local://security.permissions.net/entities#Sama...,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOME...
4,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOBBY
5,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#OFFICE
6,local://security.permissions.net/entities#Anya,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#STORAGE
7,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#KITCHEN


In [6]:
# Run the rules engine
corese.resetRuleEngine(g)
corese.loadRuleEngine(g, profile='owlrl_lite')
corese.loadRuleEngine(g, profile='owlrl_ext')
corese.loadRuleEngine(g, profile='rdfs')

: 

: 

In [ ]:
# Now that we have added our inferences, we can see the results of the manages relationships which have been inferred from both managesDepartment and mangedBy
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,s,p,o
0,local://security.permissions.net/entities#475251,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#STORAGE
1,local://security.permissions.net/entities#470799,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#OFFICE
2,local://security.permissions.net/entities#452718,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#KITCHEN
3,local://security.permissions.net/entities#487284,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#STORAGE
4,local://security.permissions.net/entities#469308,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#KITCHEN
...,...,...,...
1914,local://security.permissions.net/entities#Sama...,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOME...
1915,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOBBY
1916,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#OFFICE
1917,local://security.permissions.net/entities#Anya,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#STORAGE


In [ ]:
# Find the users who have access to a specific record
# Now that we have added our inferences, we can see the results of the manages relationships which have been inferred from both managesDepartment and mangedBy
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT *
WHERE {
?x a :User ;
    :handlesRegion/:hasCountry/:country ?y .
}
ORDER BY ?x
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,x,y
0,local://security.permissions.net/entities#Sale...,local://security.permissions.net/entities#4752...
1,local://security.permissions.net/entities#Sale...,local://security.permissions.net/entities#4688...
2,local://security.permissions.net/entities#Sale...,local://security.permissions.net/entities#4604...
3,local://security.permissions.net/entities#Sale...,local://security.permissions.net/entities#4923...
4,local://security.permissions.net/entities#Sale...,local://security.permissions.net/entities#4868...
...,...,...
154894,local://security.permissions.net/entities#Sale...,local://security.permissions.net/entities#1663...
154895,local://security.permissions.net/entities#Sale...,local://security.permissions.net/entities#4898...
154896,local://security.permissions.net/entities#Sale...,local://security.permissions.net/entities#4883...
154897,local://security.permissions.net/entities#Sale...,local://security.permissions.net/entities#9993...


In [ ]:
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT *
WHERE {
?x :hasAccessTransitive ?y .
}
ORDER BY ?x
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,x,y
0,local://security.permissions.net/entities#1010331,local://security.permissions.net/entities#HOME...
1,local://security.permissions.net/entities#1010436,local://security.permissions.net/entities#HOME...
2,local://security.permissions.net/entities#1010709,local://security.permissions.net/entities#HOME...
3,local://security.permissions.net/entities#1010898,local://security.permissions.net/entities#HOME...
4,local://security.permissions.net/entities#1011633,local://security.permissions.net/entities#HOME...
...,...,...
1906,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Sale...
1907,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Sale...
1908,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Sale...
1909,local://security.permissions.net/entities#Steve,local://security.permissions.net/entities#Sale...
